In [2]:
import pandas as pd
import yfinance as yf
from functions import process_data, read_extensions
import datetime

# Função de leitura da biblioteca Yahoo Finance
def get_yahoo_finance_data(symbol, start_date, end_date):
    try:
        data = yf.download(symbol, start=start_date, end=end_date, auto_adjust=True)
        return data
    except Exception as e:
        print(f"Erro ao buscar dados do Yahoo Finance: {e}")
        return None

# Função atualizada com o método .ffill() e yfinance
def main():
    # Na variável abaixo "data_source", escolha entre 'local' ou 'yahoo'
    data_source = 'yahoo'
    START_DATE = datetime.date(2024, 3, 1)
    END_DATE = datetime.date(2024, 3, 31)
    
    if data_source == 'yahoo':
        symbol = "^BVSP"
        data = get_yahoo_finance_data(symbol, START_DATE.strftime('%Y-%m-%d'), END_DATE.strftime('%Y-%m-%d'))
    elif data_source == 'local':
        file_path = "H:\\TCD\\_PMF\\Mod_2_Data_Handling\\00\\PETR4.csv"
        data = read_extensions.read_csv(file_path)
    else:
        print("Fonte de dados inválida.")
        return
    
    if data is not None:
        df_handled = process_data.handle_missing(data, method='ffill')
        
        # Filtro usando .loc()
        filtered_data_loc = process_data.filter_by_date(df_handled, start_date=START_DATE, end_date=END_DATE, columns=['Open', 'Close'])
        print("Exemplo de filtro por datas usando.loc():")
        print(filtered_data_loc.tail())
        
        # Calcular índices para o intervalo de datas
        start_index = df_handled.index.searchsorted(pd.to_datetime(START_DATE))
        end_index = df_handled.index.searchsorted(pd.to_datetime(END_DATE), side='right')
        
        # Filtro usando .iloc()
        filtered_data_iloc = process_data.filter_by_index(df_handled, start_index=start_index, end_index=end_index, columns=None)
        print("\nExemplo de filtro por índices usando.iloc():")
        print(filtered_data_iloc.tail())
        
        # Deslocamento de coluna com o método .shift(), dentro do mesmo intervalo de datas
        shifted_data = process_data.shift_column(filtered_data_loc, 'Close', 1)
        print("\nExemplo de deslocamento de coluna usando.shift():")
        print(shifted_data.tail())
    else:
        print("Erro ao ler os dados.")

if __name__ == "__main__":
    main()


[*********************100%%**********************]  1 of 1 completed

Exemplo de filtro por datas usando.loc():
                Open     Close
Date                          
2024-03-22  128159.0  127027.0
2024-03-25  127027.0  126931.0
2024-03-26  126931.0  126863.0
2024-03-27  126863.0  127691.0
2024-03-28  127689.0  128106.0

Exemplo de filtro por índices usando.iloc():
                Open      High       Low     Close   Volume
Date                                                       
2024-03-22  128159.0  128159.0  126879.0  127027.0  9367800
2024-03-25  127027.0  127224.0  126750.0  126931.0  8060500
2024-03-26  126931.0  127193.0  126591.0  126863.0  9435700
2024-03-27  126863.0  127756.0  126223.0  127691.0  9918200
2024-03-28  127689.0  128364.0  127270.0  128106.0  9943400

Exemplo de deslocamento de coluna usando.shift():
                Open     Close  Close_shifted_1
Date                                           
2024-03-22  128159.0  127027.0         128159.0
2024-03-25  127027.0  126931.0         127027.0
2024-03-26  126931.0  126863.0  